<a href="https://colab.research.google.com/github/SzymonNowakowski/Machine-Learning-2024/blob/master/Lab02_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab2 - unsupervisd learning: clustering
### Author: Szymon Nowakowski

In this lab, we will coduct clustering. It is a close relative of Principal Component Analysis. While PCA serves to identify directions in feature space (columns of the data matrix $X$), clustering identifies clusters of similarity in *rows* of $X$.

